## **Fase 1 - Extract**
*En esta fase inicial, el enfoque está en la extracción de datos brutos directamente desde la plataforma de Airbnb. Utilizando un scraper automatizado, extraemos información esencial sobre las propiedades disponibles, como ubicación, precios, tipo de alojamiento, disponibilidad y reseñas de los usuarios. La extracción es un paso crucial para asegurar que todos los datos relevantes se reúnan y estén listos para su análisis. Nos aseguramos de que los datos se extraigan de forma precisa y consistente, lo cual es fundamental para el éxito de las fases siguientes del proceso. El objetivo de esta fase es consolidar la información extraída en un formato accesible (DataFrame) de manera eficiente y automatizada, para que posteriormente puedan ser limpiados y transformados.* **(necesita edicion - version test)**

In [1]:
# ETL imports
import requests 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep

# General 
import pandas as pd
import numpy as np
import json          
import os
from tqdm import tqdm
        

In [2]:
opts = Options()
opts.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36")

In [3]:
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=opts)

In [4]:
# Lista de distritos de Barcelona
distritos = [
    "Ciutat Vella",
    "Eixample",
    "Sants-Montjuïc",
    "Les Corts",
    "Sarrià-Sant Gervasi",
    "Gràcia",
    "Horta-Guinardó",
    "Nou Barris",
    "Sant Andreu",
    "Sant Martí"
]

# Creamos una lista para almacenar enlaces de distritos
link_distrito = []

# URL base con el formato de Airbnb
url_base = "https://www.airbnb.es/s/{distrito}--Barcelona--España/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&monthly_start_date=2024-11-01&monthly_length=3&monthly_end_date=2025-02-01&price_filter_input_type=0&channel=EXPLORE&query={distrito}%2C%20Barcelona%2C%20España&date_picker_type=calendar&source=structured_search_input_header&search_type=autocomplete_click"

# Iterar sobre cada distrito y generar el enlace
for distrito in distritos:
    # Reemplazar espacios por %20 y caracteres especiales
    distrito_url = distrito.replace(" ", "%20").replace("à", "%C3%A0").replace("ç", "%C3%A7").replace("í", "%C3%AD")
    
    # Formatear el enlace con el nombre del distrito
    enlace = url_base.format(distrito=distrito_url)

    link_distrito.append(enlace)

# Lista vacía para almacenar los links de habitaciones
links_to_scrapp = []

# Accedemos a las paginas de cada distrito
for distrito in tqdm(link_distrito, desc="Accediendo a distritos"):
    
    driver.get(distrito)

    # Pausa
    sleep(3)

    # Bucle para la paginación y extración
    while len(links_to_scrapp) < 500:  # Limitar a 500 enlaces
        try:
            sleep(2)
            # Obtener el código fuente de la página actual
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')

            # Extraer todos los enlaces de habitaciones en la página actual
            link_habitacion = driver.find_elements(By.CSS_SELECTOR, 'a[href*="/rooms/"]')

            # Lista de urls target
            links_target = []

            # Recorrer todos los enlaces y filtrar los que contienen '/rooms'
            for _ in link_habitacion:
                link = _.get_attribute('href')
                if link and '/rooms/' in link:
                    links_target.append(link)

            # Eliminar duplicados en la lista de enlaces
            links_filtered = set(links_target)
            print(f"Número de links filtrados en la página: {len(links_filtered)}")

            # Añadimos los links en la lista de scrappeo
            for url in links_filtered:
                if len(links_to_scrapp) < 500:  # Limitar a 500 enlaces
                    links_to_scrapp.append(url)
                    print(f'Agregando URL: {url}')
                else:
                    break

            print(f"Total de links extraídos hasta ahora: {len(links_to_scrapp)}")

            if len(links_to_scrapp) >= 500:  
                print('Total de enlaces alcanzado. Deteniendo la extracción......')
                break

            # Pausa antes de hacer click en la próxima página
            sleep(2)

            # Buscar y hacer click en el botón "Siguiente"
            next_button = driver.find_element(By.CSS_SELECTOR, 'a[aria-label="Siguiente"]')
            next_button.click()
            print('Siguiente página...')

        except Exception as e:
            print('No se pudo cargar la siguiente página o no hay más páginas.')
            print(e)
            break

# Imprimir a lista final de enlaces
print("Extracción completa. Total de links:", len(links_to_scrapp))

Accediendo a distritos:   0%|          | 0/10 [00:00<?, ?it/s]

Número de links filtrados en la página: 18
Agregando URL: https://www.airbnb.es/rooms/961426648266194498?adults=1&category_tag=Tag%3A8678&children=0&enable_m3_private_room=true&infants=0&pets=0&photo_id=1880359574&search_mode=regular_search&check_in=2024-11-04&check_out=2024-11-09&source_impression_id=p3_1730033075_P3AMkQYAE6HTfQVe&previous_page_section_name=1000&federated_search_id=dcd62a08-3ba3-46ed-acc6-bb8e34069d35
Agregando URL: https://www.airbnb.es/rooms/40553667?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-11-08&check_out=2024-11-13&source_impression_id=p3_1730033075_P3IIrkbEbdDndXWT&previous_page_section_name=1000&federated_search_id=dcd62a08-3ba3-46ed-acc6-bb8e34069d35
Agregando URL: https://www.airbnb.es/rooms/729078?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2025-02-05&check_out=2025-02-10&source_impression_id=p3_1730033075_P34kvmWigKRk6Hzf&previous_page_se

Accediendo a distritos:  10%|█         | 1/10 [01:25<12:45, 85.06s/it]

No se pudo cargar la siguiente página o no hay más páginas.
Message: no such element: Unable to locate element: {"method":"css selector","selector":"a[aria-label="Siguiente"]"}
  (Session info: chrome=130.0.6723.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x009C3853+24035]
	(No symbol) [0x0094BBE4]
	(No symbol) [0x0082C2D3]
	(No symbol) [0x0086DC86]
	(No symbol) [0x0086DECB]
	(No symbol) [0x008AB9D2]
	(No symbol) [0x0088FED4]
	(No symbol) [0x008A953F]
	(No symbol) [0x0088FC26]
	(No symbol) [0x0086218C]
	(No symbol) [0x0086310D]
	GetHandleVerifier [0x00C69683+2800659]
	GetHandleVerifier [0x00CC423E+3172302]
	GetHandleVerifier [0x00CBCE52+3142626]
	GetHandleVerifier [0x00A66C00+692624]
	(No symbol) [0x00954BFD]
	(No symbol) [0x00951908]
	(No symbol) [0x00951AA0]
	(No symbol) [0x00943F50]
	BaseThreadInitThunk [0x76A67BA9+25]
	RtlInitializeExceptionChain

Accediendo a distritos:  20%|██        | 2/10 [02:34<10:07, 75.89s/it]

Número de links filtrados en la página: 18
Agregando URL: https://www.airbnb.es/rooms/1202761676097092896?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-11-23&check_out=2024-11-28&source_impression_id=p3_1730033222_P3pmJ4J4nQctXKjt&previous_page_section_name=1000&federated_search_id=3facd82b-718b-41a1-b25c-75c61013b36a
Agregando URL: https://www.airbnb.es/rooms/661475984265403325?adults=1&category_tag=Tag%3A8678&children=0&enable_m3_private_room=true&infants=0&pets=0&photo_id=1436799013&search_mode=regular_search&check_in=2024-12-11&check_out=2024-12-16&source_impression_id=p3_1730033222_P3Lz5izuEr9zHyTu&previous_page_section_name=1000&federated_search_id=3facd82b-718b-41a1-b25c-75c61013b36a
Agregando URL: https://www.airbnb.es/rooms/535507936337070255?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-11-09&check_out=2024-11-14&source_impression_id=p3_1730033222_P3BEL1zbXg

Accediendo a distritos: 100%|██████████| 10/10 [03:08<00:00, 18.85s/it]

Extracción completa. Total de links: 500


In [5]:
# Función para extraer el texto de un elemento dado
def extract_text(soup, tag, class_name, index=0, default=np.nan):
    try:
        elements = soup.find_all(tag, class_=class_name)
        return elements[index].text.strip() if elements else default  # Retorna el texto del elemento especificado
    except (AttributeError, IndexError):
        return default

# Diccionario para los campos a extraer, organizados en el orden deseado
fields = {
    "Titles": ("span", "l1h825yc", 1),  # Título del alojamiento
    "Host_name": ("div", "t1pxe1a4", 0),  # Nombre del anfitrión
    "Property_types": ("h2", "hpipapi", 0),  # Tipo de propiedad
    "Prices_per_night": ("span", "_11jcbg2", 0),  # Precio por noche
    "Check_ins": ("div", "i1303y2k", 0),  # Check-in
    "Check_outs": ("div", "i1303y2k", 1),  # Check-out
    "Cleaning_fees": ("span", "_1k4xcdh", 1),  # Tasa de limpieza
    "Location": ("div", "_152qbzi", 0),  # Ubicación
}

# Limitar a 500 enlaces para la muestra
sample_links = links_to_scrapp[:100]

# Lista para almacenar los datos extraídos
data_list = []

# Iterar sobre cada enlace en la muestra
for link in tqdm(sample_links, desc="Extrayendo datos"):

    # Asegurarse de que el enlace tenga el parámetro 'adults'
    if 'adults' not in link:
        link += '&adults=1'
    else:
        link += '?adults=1'    

    driver.get(link)  # Acceder a la página
    sleep(3)  # Espera para cargar la página

    # Obtener el objeto BeautifulSoup
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    data = {'urls': link}  # Agregar la URL al diccionario de datos

    # Extraer usando el diccionario y la función de extracción
    data.update({field: extract_text(soup, *details) for field, details in fields.items()})

    # Obtener las reseñas y calificaciones
    reviews_data = driver.find_elements(By.XPATH, '//span[@aria-hidden="true"]')
    ratings_list = []
    num_reviews_list = []
    rating = num_reviews = np.nan  # Inicializa como NaN


    # Procesar las reseñas
    for review in reviews_data:
        text = review.text

        if "(" in text:  # Verificar si el formato es el correcto
            rating = text.split(' ')[0]
            num_reviews = text.split('(')[1].replace(')', '')
            ratings_list.append(rating)  # Añadir la calificación a la lista
            num_reviews_list.append(num_reviews)  # Extraer el número de reseñas y quitar los paréntesis
            break  # Salir del bucle después de encontrar el primer formato correcto

    # Añadir calificaciones y número de reseñas al diccionario de datos
    data['Ratings'] = rating
    data['Num_reviews'] = num_reviews

    # Extracción de detalles (camas, baños, etc.) usando el selector CSS
    details = driver.find_elements(By.CSS_SELECTOR, "div.o1kjrihn ol.lgx66tx")
    details_dict = {}
    for detail in details:
        detail_text = detail.text.strip()
        # Lógica para identificar qué representa cada detalle
        if "camas" in detail_text.lower():
            details_dict['Beds'] = detail_text
        elif "dormitorios" in detail_text.lower():
            details_dict['Rooms'] = detail_text
        elif "baños" in detail_text.lower():
            details_dict['Baths'] = detail_text

    # Agregar detalles al diccionario de datos
    data.update(details_dict)

    # Agregar el número máximo de huéspedes
    n_guests = driver.find_elements(By.CSS_SELECTOR, "div[data-plugin-in-point-id='POLICIES_DEFAULT'] div.i1303y2k span")
    guests_count = np.nan  # Valor predeterminado en caso de no encontrar
    for guest in n_guests:
        if 'huésped' in guest.text or 'huéspedes' in guest.text:
            guests_count = guest.text.split()[1]  # Extraer solo el número de huéspedes
            break  # Salir del bucle si se encuentra el número de huéspedes

    # Añadir el número máximo de huéspedes al diccionario de datos
    data['Maximum_guests'] = guests_count

    # Añadir los datos extraídos a la lista
    data_list.append(data)

# Crear el DataFrame final con los datos extraídos
df = pd.DataFrame(data_list)

Extrayendo datos: 100%|██████████| 100/100 [06:27<00:00,  3.87s/it]


In [6]:
df

,urls,Titles,Host_name,Property_types,Prices_per_night,Check_ins,Check_outs,Cleaning_fees,Location,Ratings,Num_reviews,Maximum_guests,Beds,Baths,Rooms
0,https://www.airbnb.es/rooms/961426648266194498...,"LEER ANTES DE RESERVAR! Casa de huéspedes, ubi...",Quédate con Camila,"Habitación en Barcelona, España",36 €,NaN,NaN,31 €,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.airbnb.es/rooms/40553667?adults=1&...,Cama en una habitación compartida mixta con en...,Anfitrión: Lfdg Hostels,"Habitación en Barcelona, España",25 €,NaN,NaN,NaN,"Barcelona, Catalunya, España",NaN,NaN,NaN,12 camas,NaN,NaN
2,https://www.airbnb.es/rooms/729078?adults=1&ch...,Dúplex acogedor muy cerca de las Ramblas. Cerc...,Anfitrión: Flavio,Alojamiento entero: apto. residencial en Barce...,85 €,Llegada a partir de las 16:00,Salida antes de las 12:00,65 €,NaN,NaN,NaN,NaN,4 viajeros2 dormitorios3 camas,NaN,NaN
3,https://www.airbnb.es/rooms/127380575668463179...,Hola: El piso cuenta con 3 habitaciones dobles...,Anfitrión: Nic,"Alojamiento entero: apartamento en Barcelona, ...",135 €,Llegada a partir de las 15:00,Salida antes de las 11:00,60 €,"Barcelona, Catalunya, España",NaN,NaN,NaN,8 viajeros3 dormitorios6 camas,NaN,NaN
4,https://www.airbnb.es/rooms/14006090?adults=1&...,"Amplio y moderno estudio totalmente equipado, ...",Anfitrión: Anna,Habitación en apartamento con servicios en Bar...,94 €,Llegada a partir de las 14:00,Salida antes de las 11:00,NaN,"Carrer d'En Fontrodona, 15, 08004 Barcelona, S...",NaN,NaN,NaN,3 viajeros1 dormitorio3 camas,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,https://www.airbnb.es/rooms/34957866?adults=1&...,"Por favor, consulta la disponibilidad antes de...",Anfitrión: Andre,"Alojamiento entero: apartamento en Barcelona, ...",102 €,Llegada a partir de las 15:00,Salida antes de las 11:00,50 €,NaN,NaN,NaN,NaN,4 viajeros2 dormitorios4 camas,NaN,NaN
96,https://www.airbnb.es/rooms/40447482?adults=1&...,Habitación grande en un gran piso compartido.U...,Quédate con Pedro,"Habitación en Barcelona, España",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 camas,NaN,NaN
97,https://www.airbnb.es/rooms/14346513?adults=1&...,Este es nuestro increible y céntrico apartamen...,Anfitrión: Claudia,"Alojamiento entero: apartamento en Barcelona, ...",104 €,Llegada a partir de las 13:00,Salida antes de las 11:00,30 €,"Barcelona, CT, España",NaN,NaN,NaN,NaN,NaN,NaN
98,https://www.airbnb.es/rooms/125209022534921086...,"Catedral Gótica, balcón pequeña habitación p...",Quédate con Emmnuel Dario,"Habitación en Barcelona, España",47 €,Horario de llegada: de 15:00 a 20:00,Salida antes de las 10:00,NaN,"Barcelona, Catalunya, España",NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Para no correr todo el codigo, podemos basearnos en este csv y corregir lo que esta mal
df.to_csv('df_test_V1', index=False)